## What可视化

2D Grand CAM可视化模块

In [ ]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random

from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey
from onekey_algo.utils.MultiProcess import MultiProcess
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

samples = glob(r'E:\20230527-LiXueJie\patches_norm/*.jpg')
def viz_sample(samples, thread_id):
    model, transformer, device = init_from_onekey(r'E:\20230527-LiXueJie\models\inception_v3\viz')
    for n, m in model.named_modules():
        print('Feature name:', n, "|| Module:", m)
    target_layer = "Mixed_7c.branch_pool.conv"
    gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

    random.shuffle(samples)
    viz_dir = r'E:\20230527-LiXueJie/viz'
    os.makedirs(viz_dir, exist_ok=True)
    for sample in samples:
        img = default_loader(sample)
        sample_ = transformer(img)
        sample_  = sample_.view(1, *sample_.size()).to(device)
        res_cam = gradcam(x=sample_, class_idx=None)
        fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
    #     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
        axes[0].imshow(img.resize(sample_.size()[2:]))
        axes[0].axis('off')
    #     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
    #     plt.show()
    #     plt.figure(figsize=(10, 10))
    #     plt.axis('off')
        axes[1].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False))
        axes[1].axis('off')
        plt.savefig(f'{viz_dir}/{os.path.basename(sample)}', bbox_inches = 'tight')
        plt.close()
viz_sample(samples, thread_id=0)